In [ ]:
#in google Earth Engine command line run this code
# gcloud projects list

In [2]:
!pip -q install geemap geopandas

from google.colab import drive
drive.mount('/content/drive')

import ee, geemap, datetime

# Initialize EE (assumes you've enabled Earth Engine API on this project)
PROJECT = "geeday2"  # <-- change if needed
ee.Initialize(project=PROJECT)
print("EE initialized with project:", PROJECT)



Mounted at /content/drive
EE initialized with project: geeday2


In [3]:
# Install deps for reading shapefiles and converting to EE
!pip -q install geemap geopandas shapely fiona pyproj pycrs rtree
# 1) Load boundary from your shapefile on Drive
BOUNDARY_SHP = r"/content/drive/MyDrive/Urban Data Science/Day 1/Data/Linz Statistical districts/Statistische_Bezirke_20250428.shp"

# Convert shapefile -> ee.FeatureCollection (needs geopandas; geemap handles it)
aoi_fc = geemap.shp_to_ee(BOUNDARY_SHP)
aoi = aoi_fc.geometry()  # union geometry for the whole AOI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 25.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Field Stand create as date field, though DateTime requested.
  ogr_write(


In [5]:
# 2) Landsat 8 C2 L2: dates + cloud filter
START_DATE = "2023-01-01"
END_DATE   = "2023-12-31"
MAX_CLOUD  = 20  # percent (without % sign)

# Mask clouds/shadows/snow via QA_PIXEL and scale SR bands to reflectance
# reflectance = DN * 0.0000275 - 0.2  (USGS C2 L2)
def mask_scale_l8(img):
    qa = img.select('QA_PIXEL')
    cloud     = qa.bitwiseAnd(1 << 3).eq(0)
    cloudshad = qa.bitwiseAnd(1 << 4).eq(0)
    snow      = qa.bitwiseAnd(1 << 5).eq(0)
    mask = cloud.And(cloudshad).And(snow)

    optical = (img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'])
                 .multiply(0.0000275).add(-0.2))
    return img.addBands(optical, None, True).updateMask(mask)

landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(START_DATE, END_DATE)
           .filter(ee.Filter.lt('CLOUD_COVER', MAX_CLOUD))
           .map(mask_scale_l8))

In [6]:

# 3) Print image count
count = landsat.size().getInfo()
print("Number of images:", count)



Number of images: 18


In [7]:
# 4) Print dates of images
ts = landsat.aggregate_array('system:time_start').getInfo()  # list of epoch ms
dates = [datetime.datetime.utcfromtimestamp(t/1000).strftime('%Y-%m-%d') for t in ts]
print("Image dates:", dates)



Image dates: ['2023-03-01', '2023-03-17', '2023-05-04', '2023-07-23', '2023-08-24', '2023-09-09', '2023-10-11', '2023-11-12', '2023-03-17', '2023-05-04', '2023-06-21', '2023-07-23', '2023-08-24', '2023-09-09', '2023-10-11', '2023-05-27', '2023-06-12', '2023-08-15']


In [8]:
# 5) Median composite and clip
composite = landsat.median().clip(aoi)



In [9]:
# 6) Select bands (scaled reflectance bands)
bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
input_image = composite.select(bands)



In [10]:
# 7) Visualize true color (reflectance 0–0.3 after scaling)
geemap.ee_initialize(project=PROJECT)
m = geemap.Map()
m.centerObject(aoi, 10)
m.addLayer(composite, {"bands": ["SR_B4","SR_B3","SR_B2"], "min": 0, "max": 0.3},
           "Raw Composite (true color, 2023)")
m

Map(center=[48.284323901222535, 14.316153339534392], controls=(WidgetControl(options=['position', 'transparent…

In [21]:
# 8) Compute NDBI (SWIR1 vs NIR)
ndbi = composite.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')

# 9) Mask for built-up areas (suggested threshold: 0.1)
built_up = ndbi.gt(0.1).selfMask()

#import geemap

#geemap.ee_initialize(project=PROJECT)   # or omit if already initialized
m = geemap.Map()
m.centerObject(aoi, 10)

# Continuous NDBI layer: values ~[-1, 1]
m.addLayer(
    ndbi,
    {"min": -0.5, "max": 0.5, "palette": ["#2c7fb8", "#ffffbf", "#d7301f"]},
    "NDBI"
)

# Legends
# 1) NDBI (approximate gradient using your 3-color palette)
legend_ndbi = {
    "-0.5 (low)": "#2c7fb8",
    "0.0":        "#ffffbf",
    "0.5 (high)": "#d7301f",
}
m.add_legend(title="NDBI", legend_dict=legend_ndbi, position="bottomleft")

# 2) Built-up mask legend
m.add_legend(
    title="Built-up mask",
    labels=["NDBI > 0.1"],
    colors=["#ff0000"],
    position="bottomright"
)

# Optional: layer control
#m.add_layer_control()
# Optional mask overlay for built-up
m.addLayer(built_up, {"palette": ["#ff0000"]}, "Built-up (NDBI > 0.1)")

m


Map(center=[48.28432390157587, 14.316153340320216], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
# Make sure 'composite' exists (your Landsat median composite)
bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
input_img = composite.select(bands)


In [15]:
#Preparing the training dataset
# Assumes: aoi, input_img, bands already defined (as in your previous steps)
import ee
START_DATE = "2023-01-01"
END_DATE   = "2023-12-31"

# 1) Build a DW label mosaic (mode) over your period & AOI
dw = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
      .filterDate(START_DATE, END_DATE)
      .filterBounds(aoi))
dw_label = dw.select('label').mode().clip(aoi)

# 2) Remap DW classes to your 7-class scheme/order:
# Your order: 0 BuiltUps, 1 Agriculture, 2 Water, 3 Bushlands, 4 Barelands, 5 Wetlands, 6 Forests
# DW label codes: 0 water, 1 trees, 2 grass, 3 flooded veg, 4 crops, 5 shrub/scrub, 6 built, 7 bare, 8 snow/ice
src = [6, 4, 0, 5, 7, 3, 1]   # DW: built, crops, water, shrub, bare, flooded, trees
dst = [0, 1, 2, 3, 4, 5, 6]   # your class ids
label7 = dw_label.remap(src, dst, 255)  # 255 = "ignore"
label7 = label7.updateMask(label7.neq(255))

# 3) Stratified sampling to create a training set
training = (input_img.addBands(label7.rename('Class'))
            .stratifiedSample(
                numPoints=0,                 # use classPoints below
                classBand='Class',
                region=aoi,
                scale=30,                    # Landsat scale
                classValues=[0,1,2,3,4,5,6],
                classPoints=[800]*7,         # ~800 samples per class (tune)
                seed=42,
                geometries=False))

# 4) Split train/test
split = training.randomColumn('rand', seed=42)
train_set = split.filter(ee.Filter.lt('rand', 0.8))
test_set  = split.filter(ee.Filter.gte('rand', 0.8))

# 5) Train RF and classify
classifier = (ee.Classifier.smileRandomForest(numberOfTrees=200, seed=42)
              .train(features=train_set, classProperty='Class', inputProperties=bands))

classified = input_img.classify(classifier)
smoothed   = classified.focal_mode(5, 'square', 'pixels')

# 6) Accuracy (vs DW-derived test labels)
cm = test_set.classify(classifier).errorMatrix('Class', 'classification')
print("Confusion Matrix:\n", cm.getInfo())
print("Overall Accuracy:", cm.accuracy().getInfo())

# 7) Map
!pip -q install geemap
import geemap
geemap.ee_initialize()

palette = [
  '#FFA500', # 0 BuiltUps
  '#ADFF2F', # 1 Agriculture
  '#1E90FF', # 2 Water
  '#FFFF00', # 3 Bushlands
  '#000000', # 4 Barelands
  '#0000FF', # 5 Wetlands
  '#054907'  # 6 Forests
]


# Legend (matches your class IDs 0..6 and the same palette)
legend_labels = [
    "BuiltUps", "Agriculture", "Water",
    "Bushlands", "Barelands", "Wetlands", "Forests"
]
legend_colors = [
    "#FFA500", "#ADFF2F", "#1E90FF",
    "#FFFF00", "#000000", "#0000FF", "#054907"
]

# Add legend and a layer control



m = geemap.Map()
m.centerObject(aoi, 10)
m.addLayer(classified, {'min': 0, 'max': 6, 'palette': palette}, 'RF from DynamicWorld labels')
m.addLayer(smoothed,   {'min': 0, 'max': 6, 'palette': palette}, 'Smoothed')
m.add_legend(title="LULC (RF)", labels=legend_labels, colors=legend_colors, position="bottomright")
m.add_layer_control()
m


Confusion Matrix:
 [[113, 12, 0, 1, 17, 0, 7], [15, 120, 0, 0, 0, 0, 9], [8, 1, 143, 0, 5, 0, 4], [1, 0, 0, 0, 0, 0, 0], [9, 2, 2, 0, 148, 0, 0], [0, 0, 0, 0, 0, 0, 0], [3, 12, 2, 0, 0, 0, 141]]
Overall Accuracy: 0.8580645161290322


Map(center=[48.28432390157587, 14.316153340320216], controls=(WidgetControl(options=['position', 'transparent_…

In [16]:
# --- Install deps (quiet) ---
!pip -q install "shapely>=2.0.1" geopandas geemap fiona pyproj rtree pycrs

from google.colab import drive
drive.mount('/content/drive')  # ok if already mounted

import ee, geemap, geopandas as gpd, datetime
from shapely.validation import make_valid
from shapely.ops import transform, unary_union

# --- Initialize EE ---
PROJECT = "geeday2"   # <-- your GCP project ID
ee.Initialize(project=PROJECT)
print("EE initialized with:", PROJECT)

# --- 1) Load & clean AOI from shapefile ---
SHP = r"/content/drive/MyDrive/Urban Data Science/Day 1/Data/Linz Statistical districts/Statistische_Bezirke_20250428.shp"

gdf = gpd.read_file(SHP)
if gdf.empty:
    raise ValueError("Shapefile loaded but contains no features.")

# Keep only non-empty polygons
gdf = gdf[gdf.geometry.notna() & (~gdf.geometry.is_empty)]
gdf = gdf[gdf.geom_type.isin(["Polygon","MultiPolygon"])]

# Ensure WGS84
gdf = (gdf.set_crs(epsg=4326) if gdf.crs is None else gdf.to_crs(epsg=4326))

# Make geometries valid and drop Z dimension (if present)
def drop_z(geom):
    try:
        return transform(lambda x, y, z=None: (x, y), geom)
    except Exception:
        return geom

gdf["geometry"] = gdf.geometry.apply(make_valid).apply(drop_z)

# Dissolve to a single boundary (simplifies conversion)
aoi_geom = unary_union(gdf.geometry.tolist())
aoi_gdf  = gpd.GeoDataFrame(geometry=[aoi_geom], crs="EPSG:4326")

# Convert to EE
aoi_fc = geemap.gdf_to_ee(aoi_gdf)
aoi    = aoi_fc.geometry()
print("AOI bounds:", aoi.bounds().getInfo())

# --- 2) Landsat 8 C2 L2: mask + scale + composite ---
START_DATE = "2023-01-01"
END_DATE   = "2023-12-31"
MAX_CLOUD  = 20  # percent (no % sign)

# L2 scaling: reflectance = DN*0.0000275 - 0.2; QA_PIXEL mask: cloud/shadow/snow
def mask_scale_l8(img):
    qa = img.select('QA_PIXEL')
    cloud     = qa.bitwiseAnd(1 << 3).eq(0)
    cloudshad = qa.bitwiseAnd(1 << 4).eq(0)
    snow      = qa.bitwiseAnd(1 << 5).eq(0)
    mask = cloud.And(cloudshad).And(snow)

    optical = (img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'])
                 .multiply(0.0000275).add(-0.2))
    return img.addBands(optical, None, True).updateMask(mask)

landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(START_DATE, END_DATE)
           .filter(ee.Filter.lt('CLOUD_COVER', MAX_CLOUD))
           .map(mask_scale_l8))

composite = landsat.median().clip(aoi)

# --- 3) Define input_img for later steps (e.g., indices/classification) ---
bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
input_img = composite.select(bands)
print("Ready. input_img bands:", bands)

# (Optional) Quick sanity checks
count = landsat.size().getInfo()
print("Images in collection:", count)
ts = landsat.aggregate_array('system:time_start').getInfo()
print("First/last dates:", (datetime.datetime.utcfromtimestamp(ts[0]/1000).strftime('%Y-%m-%d'),
                            datetime.datetime.utcfromtimestamp(ts[-1]/1000).strftime('%Y-%m-%d')))

# (Optional) Quick map
try:
    geemap.ee_initialize(project=PROJECT)
    m = geemap.Map()
    m.centerObject(aoi, 10)
    m.addLayer(composite, {"bands":["SR_B4","SR_B3","SR_B2"], "min":0, "max":0.3}, "Composite (true color)")
    display(m)
except Exception as e:
    print("Map preview skipped:", e)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
EE initialized with: geeday2
AOI bounds: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[14.245719990613695, 48.211371163335635], [14.409216854384134, 48.211371163335635], [14.409216854384134, 48.37869258551145], [14.245719990613695, 48.37869258551145], [14.245719990613695, 48.211371163335635]]]}
Ready. input_img bands: ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
Images in collection: 18
First/last dates: ('2023-03-01', '2023-08-15')


Map(center=[48.28432390157587, 14.316153340320216], controls=(WidgetControl(options=['position', 'transparent_…